In [12]:
class LinkedListNode:
    def __init__(self, key, val1 = None, val2 = None):
        self.key = key
        self.val1 = val1
        self.val2 = val2
        self.next = None

In [13]:
class LinkedList:
    def __init__(self):
        self.head = None
        # self.count = 0
    def update(self, key, val1, val2):
        exist = False
        node = LinkedListNode(key, val1, val2)
        if not self.head:
            self.head = node
            return exist
        current = self.head
        while True:
            if current.key == key:
                exist = True
                current.val1 = val1 if val1 else current.val1
                current.val2 = val2 if val2 else current.val2
                return exist
            if not current.next:
                break
            current = current.next
        current.next = node
        return exist
    def remove(self, key):
        if not self.head or (not self.head.next and self.head.key != key):
            return False
        if self.head.key == key:
            self.head = self.head.next
            return True
        current = self.head
        while current.next:
            if current.next.key == key:
                current.next = current.next.next
                return True
            current = current.next
        return False
    def search(self, key):
        if not self.head or (not self.head.next and self.head.key != key):
            return None, None
        if self.head.key == key:
            return self.head.val1, self.head.val2
        current = self.head
        while current.next:
            if current.next.key == key:
                return current.val1, current.val2
            current = current.next
        return None, None
    def __str__(self):
        current = self.head
        str_repr = []
        while current:
            str_repr.append(f'({current.key}, {current.val1}, {current.val2})')
            current = current.next
        return ' '.join(str_repr)

In [14]:
class HashTable: # collision handled by chaining
    def __init__(self, capacity):
        self.array = []
        self.capacity = capacity # num of buckets
        for i in range(capacity):
            ll = LinkedList()
            self.array.append(ll)
    def _hashing(self, key): # hash function
        return sum([ord(c) for c in key]) % self.capacity
    def insert(self, key, val1, val2):
        idx = self._hashing(key)
        return self.array[idx].update(key, val1, val2)
    def delete(self, key):
        idx = self._hashing(key)
        if self.array[idx].remove(key):
            return True
        return False
    def search(self, key):
        idx = self._hashing(key)
        return self.array[idx].search(key)
    def show(self):
        print(self)
    def update_vals(self, vals):
        if vals[0]:
            print('Not implemented')
        if vals[1]:
            for chain in self.array:
                if not chain: continue
                node = chain.head
                while node:
                    node.val2 += vals[1]
                    node = node.next
    def __str__(self):
        str_repr = []
        for bucket, chain in enumerate(self.array):
            str_repr.append(f'Bucket {bucket}: {str(chain)}')
        return '\n'.join(str_repr)

In [15]:
class BSTNode:
    def __init__(self, key):
        self.key = key
        self.items = []
        self.left = None
        self.right = None
    def __str__(self):
        return f'{self.key}, {self.items}'

In [16]:
import copy
class BST:
    def __init__(self):
        self.root = None
    def __str__(self):
        if not self.root:
            return ''
        str_repr = []
        str_repr.append(BST._str(self.root.left))
        str_repr.append(f'Level {self.root.key} {self.root.items}\n')
        str_repr.append(BST._str(self.root.right))
        return ''.join(str_repr)
    @staticmethod
    def _str(node): # recursive function called by __str__()
        if not node:
            return ''
        str_repr = []
        str_repr.append(BST._str(node.left))
        str_repr.append(f'Level {node.key} {node.items}\n')
        str_repr.append(BST._str(node.right))
        return ''.join(str_repr)
    @staticmethod
    def _insert(node, key, val): # recursive function called by insert()
        if not node:
            node = BSTNode(key)
            node.items.append(val)
        elif node.key == key:
            node.items.append(val)
        elif node.key > key:
            node.left = BST._insert(node.left, key, val)
        else:
           node.right = BST._insert(node.right, key, val)
        return node        
    @staticmethod
    def _delete_key(node, key): # recursive function called by delete_key()
        if not node:
            return None
        if node.key == key:
            if not node.right and not node.left:
                return None
            if not node.right:
                return node.left
            if not node.left:
                return node.right
            current = node.right
            if current.left:
                while current.left:
                    prev = current
                    current = current.left
                node.key, node.items = current.key, copy.deepcopy(current.items)
                prev.left = current.right
            else:
                current.left = node.left
                node = current
        elif node.key < key:
            node.right = BST._delete_key(node.right, key)
        else:
            node.left = BST._delete_key(node.left, key)
        return node
    @staticmethod
    def _delete_item(node, item): # recursive function called by delete_item()
        item_found = False
        if not node:
            return None, item_found
        if item in node.items:
            item_found = True
            node.items.remove(item)
            if not node.items:
                return BST._delete_key(node, node.key), item_found
            return node, item_found
        # if node.right:
        node.right, item_found = BST._delete_item(node.right, item)
        # if not item_found, then try the left child:
        if not item_found:
            node.left, item_found = BST._delete_item(node.left, item)
        return node, item_found
    @staticmethod
    def _update_key(node, inc): # recursive function called by update_key()
        if not node:
            return
        node.key += inc
        BST._update_key(node.right, inc)
        BST._update_key(node.left, inc)
    @staticmethod
    def _search(node, key): # recursive function called by search()
        if not node:
            return []
        if node.key == key:
            return node.items
        if node.key < key:
            return BST._search(node.right, key)
        return BST._search(node.left, key)
    def insert(self, key, item):
        self.root = BST._insert(self.root, key, item)
    def delete_key(self, key):
        self.root = BST._delete_key(self.root, key)
    def delete_item(self, item):
        self.root, _ = BST._delete_item(self.root, item)
    def update_key(self, inc):
        BST._update_key(self.root, inc)
    def search(self, key):
        return BST._search(self.root, key)

In [17]:
import random
class AnimalManagementSystem:
    FCLTY_LEVELS = {'Intensive': (8, 10),
                    'Advanced': (4, 7),
                    'Basic': (1, 3)
                   }
    CARE_LEVEL_LOW = 1
    CARE_LEVEL_HIGH = 10
    CARE_LEVEL_INC = 1
    FACILITY_CLOSURE = 2
    NUM_DATABASE_BUCKETS = 7
    def __init__(self):
        self.database = HashTable(AnimalManagementSystem.NUM_DATABASE_BUCKETS)
        self.careMgmt = BST()
        self.cycle_count = 1
    def _update_animal(self, name, species, care_level):
        if self.database.insert(name, species, care_level):
            self.careMgmt.delete_item(name)
        self.careMgmt.insert(care_level, name)
    def insert_by_name(self, name, species, care_level):
        self._update_animal(name, species, care_level)
        print(f'({name}, {species}, {care_level}) inserted')
    def delete_by_name(self, name, *_):
        if self.database.delete(name):
            self.careMgmt.delete_item(name)
            print(f'{name} deleted')
        else:
            print(f'{name} not found')
    def search_by_name(self, name, *_):
        species, care_level = self.database.search(name)
        if not care_level:
            print(f'{name} not found')
            return
        animal_name = 'Animal name'
        print(f'{animal_name}: {name}')
        if species:
            print(f'{'Species':>{len(animal_name)}}: {species}')
        print(f'{'Care Level':>{len(animal_name)}}: {care_level}')
    def insert_by_care_level(self, name, _, care_level):
        self._update_animal(name, None, care_level)
        print(f'({name}, {care_level}) inserted')
    def show_database(self, *_):
        self.database.show()
    def show_by_care_level(self, _1, _2, care_level):
        if care_level == 0:
            print(self.careMgmt)
        else:
            print(self.careMgmt.search(care_level))
    def _provide_care(self):
        names_cared = []
        for care, level in AnimalManagementSystem.FCLTY_LEVELS.items():
            # iterate over facilities of all care levels
            lo, hi = int(level[0]), int(level[1]) # range of the care level
            for l in range(hi, lo-1, -1): # iterate over the care level range
                names = self.careMgmt.search(l)
                if names: # if there are animals at this care level
                    print(f'{care} care facility admitting {names}')
                    names_cared += names
                    self.careMgmt.delete_key(l)
                    break
            else: # if no animal found within this range
                print(f'No animal is in need of {care} care.')
        for name in names_cared: # re-assign care level to all animals admitted this round
            # species, _ = self.database.search(name)
            care_level = random.randint(AnimalManagementSystem.CARE_LEVEL_LOW, AnimalManagementSystem.CARE_LEVEL_HIGH-1)
            self._update_animal(name, None, care_level)
    def _increase_care_level(self):
        print('Increasing care level')
        self.database.update_vals((None, AnimalManagementSystem.CARE_LEVEL_INC))
        self.careMgmt.update_key(AnimalManagementSystem.CARE_LEVEL_INC)
    def simulate_providing_care(self, *_):
        cont = True
        while cont: # increment care level every cycle_count cycles until 'not continued'
            if self.cycle_count:
                self._provide_care()
            else:
                self._increase_care_level()
            ans = input('Continue? (y|Y for yes, anything else for no): ')
            self.cycle_count = (self.cycle_count + 1) % AnimalManagementSystem.FACILITY_CLOSURE
            cont = True if ans in ['Y', 'y'] else False

In [18]:
def invalid_choice(*_):
    print('Invalid choice!\n')

In [19]:
def enter_name(i):
    while True:
        try:
            name = input(f'Enter animal name (<Enter> = auto-populated):')
            assert name.replace(" ", "").isalnum() or name == '', 'Alphanumerics or <Enter> please'
            if name == '': # auto populate animal info
                return f'Animal{i}', f'Species{i}', i % Care_Level_Max + 1, i+1
            return name, None, None, i
        except AssertionError as msg:
            print(msg)

In [20]:
def enter_species():
    while True:
        try:
            species = input('Enter species:')
            assert species.replace(" ", "").isalpha() and species, 'Alphabets please'
            return species
        except AssertionError as msg:
            print(msg)

In [21]:
def enter_care_level(has_default=False):
    if has_default:
        prompt = f'Enter care level (1-{Care_Level_Max}, <Enter> = All levels)' + ':'
    else:
        prompt = f'Enter care level (1-{Care_Level_Max})' + ':'
    all = False
    while True:
        try:
            care_level = input(prompt)
            if has_default and care_level == '':
                care_level = 0
            else:
                care_level = int(care_level)
                assert 10 >= care_level >= 1
            return care_level
        except (ValueError, AssertionError):
            print('Invalid input')

In [22]:
Care_Level_Max = 10
MENU_INSERT_BY_NAME = '1'
MENU_DELETE_BY_NAME = '2'
MENU_SEARCH_BY_NAME = '3'
MENU_INSERT_BY_LEVEL = '4'
MENU_SHOW_DATABASE = '5'
MENU_SHOW_BY_LEVEL = '6'
MENU_SIM_CARE_SYS = '7'
system = AnimalManagementSystem()
menu = {MENU_INSERT_BY_NAME: ('Insert an animal by name', system.insert_by_name),
        MENU_DELETE_BY_NAME: ('Delete an animal by name', system.delete_by_name),
        MENU_SEARCH_BY_NAME: ('Search an animal by name', system.search_by_name),
        MENU_INSERT_BY_LEVEL: ('Insert an animal by care level', system.insert_by_care_level),
        MENU_SHOW_DATABASE: ('Show animal database', system.show_database),
        MENU_SHOW_BY_LEVEL: ('Show animals by care level', system.show_by_care_level),
        MENU_SIM_CARE_SYS: ('Simulate care system', system.simulate_providing_care),
        'q': ('Quit', None)
       }
print('Welcome to the Animal Management System!')
i = 0
while True:
    for m in menu: # display menu
        print(f'{m}: {menu[m][0]}')
    choice = input(f'Please choose an operation:')
    if choice == 'q':
        break
    name = species = care_level = None
    if choice  == MENU_INSERT_BY_NAME:
        name, species, care_level, i = enter_name(i)
        if not species:
            species = enter_species()
            care_level = enter_care_level()
    elif choice in [MENU_DELETE_BY_NAME, MENU_SEARCH_BY_NAME]:
        name, _, _, _ = enter_name(i)
    elif choice == MENU_INSERT_BY_LEVEL:
        care_level = enter_care_level()
        name, _, _, i = enter_name(i)
    elif choice == MENU_SHOW_BY_LEVEL:
        care_level = enter_care_level(True)
    # if valid choice, call the function stored in the 2nd element of the tuple
    # if invalid choice, call invalid_choice()
    menu.get(choice, (None, invalid_choice))[1](name, species, care_level)
    print('\n', end='')    

Welcome to the Animal Management System!
1: Insert an animal by name
2: Delete an animal by name
3: Search an animal by name
4: Insert an animal by care level
5: Show animal database
6: Show animals by care level
7: Simulate care system
q: Quit


Please choose an operation: 1
Enter animal name (<Enter> = auto-populated): 


(Animal0, Species0, 1) inserted

1: Insert an animal by name
2: Delete an animal by name
3: Search an animal by name
4: Insert an animal by care level
5: Show animal database
6: Show animals by care level
7: Simulate care system
q: Quit


Please choose an operation: 1
Enter animal name (<Enter> = auto-populated): 


(Animal1, Species1, 2) inserted

1: Insert an animal by name
2: Delete an animal by name
3: Search an animal by name
4: Insert an animal by care level
5: Show animal database
6: Show animals by care level
7: Simulate care system
q: Quit


...

Please choose an operation: 5


Bucket 0: (Animal2, Species2, 3)
Bucket 1: (Animal3, Species3, 4)
Bucket 2: (Animal4, Species4, 5)
Bucket 3: (Animal5, Species5, 6)
Bucket 4: 
Bucket 5: (Animal0, Species0, 1)
Bucket 6: (Animal1, Species1, 2)

1: Insert an animal by name
2: Delete an animal by name
3: Search an animal by name
4: Insert an animal by care level
5: Show animal database
6: Show animals by care level
7: Simulate care system
q: Quit


Please choose an operation: 6
Enter care level (1-10, <Enter> = All levels): 


Level 1 ['Animal0']
Level 2 ['Animal1']
Level 3 ['Animal2']
Level 4 ['Animal3']
Level 5 ['Animal4']
Level 6 ['Animal5']


1: Insert an animal by name
2: Delete an animal by name
3: Search an animal by name
4: Insert an animal by care level
5: Show animal database
6: Show animals by care level
7: Simulate care system
q: Quit


Please choose an operation: q
